# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

[Te proporcionamos algunos comentarios para orientarte mientras completas este proyecto. Pero debes asegurarte de eliminar todos los comentarios entre corchetes antes de entregar tu proyecto.]

[Antes de sumergirte en el análisis de datos, explica por tu propia cuenta el propósito del proyecto y las acciones que planeas realizar.]

[Ten en cuenta que estudiar, modificar y analizar datos es un proceso iterativo. Es normal volver a los pasos anteriores y corregirlos/ampliarlos para permitir nuevos pasos.]

## Inicialización

In [16]:
# Cargar todas las librerías
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()


## Cargar datos

In [17]:
# Carga los archivos de datos en diferentes DataFrames
calls = pd.read_csv("megaline_calls.csv")
internet = pd.read_csv("megaline_internet.csv")
messages = pd.read_csv("megaline_messages.csv")
plans = pd.read_csv("megaline_plans.csv")
users = pd.read_csv("megaline_users.csv")


In [18]:
def looks(datos):
    print("describe:")
    print(datos.describe(include= "all"))
    print()
    print("info:")
    print(datos.info())
    print()
    print("Head:")
    print(datos.head())
    print()
    print("NA?:")
    print(datos.isna().sum())
    print()
    print("Duplicados:")
    print(datos.duplicated().sum())
    
    

## Preparar los datos

[Los datos para este proyecto se dividen en varias tablas. Explora cada una para tener una comprensión inicial de los datos. Si es necesario, haz las correcciones requeridas en cada tabla.]

### Tarifas

In [19]:
# Imprime una muestra de los datos para las tarifas
looks(plans)


describe:
        messages_included  mb_per_month_included  minutes_included  \
count            2.000000               2.000000          2.000000   
unique                NaN                    NaN               NaN   
top                   NaN                    NaN               NaN   
freq                  NaN                    NaN               NaN   
mean           525.000000           23040.000000       1750.000000   
std            671.751442           10861.160159       1767.766953   
min             50.000000           15360.000000        500.000000   
25%            287.500000           19200.000000       1125.000000   
50%            525.000000           23040.000000       1750.000000   
75%            762.500000           26880.000000       2375.000000   
max           1000.000000           30720.000000       3000.000000   

        usd_monthly_pay  usd_per_gb  usd_per_message  usd_per_minute plan_name  
count          2.000000     2.00000         2.000000        2.000000

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

#### Corregir datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

 No existen datos que haya que corregir ya que son solo dos tarifas y los datos de los mismos

#### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [20]:
plans["gb_per_month"] =  plans["mb_per_month_included"]/1000
#se añade una columa con los GB por mes, para hacer mas manejables los datos y faciles de entender

## Usuarios/as

In [21]:
# Imprime la información general/resumida sobre el DataFrame de usuarios
looks(users)


describe:
            user_id first_name last_name         age  \
count    500.000000        500       500  500.000000   
unique          NaN        458       399         NaN   
top             NaN    Leonila     David         NaN   
freq            NaN          3         3         NaN   
mean    1249.500000        NaN       NaN   45.486000   
std      144.481833        NaN       NaN   16.972269   
min     1000.000000        NaN       NaN   18.000000   
25%     1124.750000        NaN       NaN   30.000000   
50%     1249.500000        NaN       NaN   46.000000   
75%     1374.250000        NaN       NaN   61.000000   
max     1499.000000        NaN       NaN   75.000000   

                                             city    reg_date  plan  \
count                                         500         500   500   
unique                                         73         266     2   
top     New York-Newark-Jersey City, NY-NJ-PA MSA  2018-03-08  surf   
freq                             

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

 Se observa que solo 34 usuarios cancelaron su plan, como indican los ausentes en _churn_date_. Ademas se observa que la columna reg_date es un string, se decide dejarlo asi de momento a menos que se requieran operaciones que deban tener en cuenta el tiempo. 

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [22]:
users["reg_date"] = pd.to_datetime(users["reg_date"])
users["churn_date"] = pd.to_datetime(users["churn_date"])

## Llamadas

In [39]:
# Imprime la información general/resumida sobre el DataFrame de las llamadas
looks(calls)

describe:
             id        user_id                      call_date       duration  \
count    137735  137735.000000                         137735  137735.000000   
unique   137735            NaN                            NaN            NaN   
top     1000_93            NaN                            NaN            NaN   
freq          1            NaN                            NaN            NaN   
mean        NaN    1247.658046  2018-09-25 19:57:56.319018496       6.745927   
min         NaN    1000.000000            2018-01-15 00:00:00       0.000000   
25%         NaN    1128.000000            2018-08-08 00:00:00       1.290000   
50%         NaN    1247.000000            2018-10-12 00:00:00       5.980000   
75%         NaN    1365.000000            2018-11-26 00:00:00      10.690000   
max         NaN    1499.000000            2018-12-31 00:00:00      37.600000   
std         NaN     139.416268                            NaN       5.839241   

                month        

In [24]:
def redondeo(dur):
    red = np.ceil(dur)
    return int(red)

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

In [25]:
calls["call_date"] = pd.to_datetime(calls["call_date"], format= "%Y-%m-%d")
calls["month"] = calls["call_date"].dt.month

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [46]:
calls["duracion_redond"] = calls["duration"].apply(redondeo)


## Mensajes

In [27]:
# Imprime la información general/resumida sobre el DataFrame de los mensajes
looks(messages)


describe:
              id       user_id message_date
count      76051  76051.000000        76051
unique     76051           NaN          351
top     1000_125           NaN   2018-12-28
freq           1           NaN          702
mean         NaN   1245.972768          NaN
std          NaN    139.843635          NaN
min          NaN   1000.000000          NaN
25%          NaN   1123.000000          NaN
50%          NaN   1251.000000          NaN
75%          NaN   1362.000000          NaN
max          NaN   1497.000000          NaN

info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB
None

Head:
         id  user_id message_date
0  1000_125     1000   2018-12-27
1  1000

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [28]:
messages["message_date"] = pd.to_datetime(messages["message_date"])
messages["month"] = messages["message_date"].dt.month

## Internet

In [29]:
# Imprime la información general/resumida sobre el DataFrame de internet
looks(internet)


describe:
             id        user_id session_date        mb_used
count    104825  104825.000000       104825  104825.000000
unique   104825            NaN          351            NaN
top     1000_13            NaN   2018-12-24            NaN
freq          1            NaN          851            NaN
mean        NaN    1242.496361          NaN     366.713701
std         NaN     142.053913          NaN     277.170542
min         NaN    1000.000000          NaN       0.000000
25%         NaN    1122.000000          NaN     136.080000
50%         NaN    1236.000000          NaN     343.980000
75%         NaN    1367.000000          NaN     554.610000
max         NaN    1499.000000          NaN    1693.470000

info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-nul

[Describe lo que ves y observas en la información general y en la muestra de datos impresa para el precio de datos anterior. ¿Hay algún problema (tipos de datos no adecuados, datos ausentes, etc.) que pudieran necesitar investigación y cambios adicionales? ¿Cómo se puede arreglar?]

### Corregir los datos

[Corrige los problemas obvios con los datos basándote en las observaciones iniciales.]

### Enriquecer los datos

[Agrega factores adicionales a los datos si crees que pudieran ser útiles.]

In [30]:
internet["session_date"] = pd.to_datetime(internet["session_date"])
internet["gb_used"] = internet["mb_used"]/1000
internet["month"] = internet["session_date"].dt.month

## Estudiar las condiciones de las tarifas

[Es sumamente importante entender cómo funcionan las tarifas, cómo se les cobra a los usuarios en función de su plan de suscripción. Así que te sugerimos imprimir la información de la tarifa para ver una vez más sus condiciones.]

In [31]:
# Imprime las condiciones de la tarifa y asegúrate de que te quedan claras
plans


,messages_included,mb_per_month_included,minutes_included,usd_monthly_pay,usd_per_gb,usd_per_message,usd_per_minute,plan_name,gb_per_month
0,50,15360,500,20,10,0.03,0.03,surf,15.36
1,1000,30720,3000,70,7,0.01,0.01,ultimate,30.72


## Agregar datos por usuario

[Ahora que los datos están limpios, agrega los datos por usuario y por periodo para que solo haya un registro por usuario y por periodo. Esto facilitará mucho el análisis posterior.]

In [51]:
# Calcula el número de llamadas hechas por cada usuario al mes. Guarda el resultado.
cals_per_user = calls.pivot_table(index= ["user_id","month"],values= "duracion_redond", aggfunc= ["sum", "count" ])
cals_per_user.columns = ["minutos_consumidos", "total_llamadas"]
cals_per_user.reset_index(inplace=True)


In [52]:
# Calcula la cantidad de minutos usados por cada usuario al mes. Guarda el resultado.
cals_per_user


,user_id,month,minutos_consumidos,total_llamadas
0,1000,12,124,16
1,1001,8,182,27
2,1001,9,315,49
3,1001,10,393,65
4,1001,11,426,64
...,...,...,...,...
2253,1498,12,339,39
2254,1499,9,346,41
2255,1499,10,385,53
2256,1499,11,308,45


In [63]:
# Calcula el número de mensajes enviados por cada usuario al mes. Guarda el resultado.
msj_mth = messages.pivot_table(index= ["user_id","month"], values= "id", aggfunc= "count" ) #groupby(["user_id","month"])["id"].count()
msj_mth.columns = ["mensajes_enviados"]
msj_mth.reset_index(inplace=True)
msj_mth


,user_id,month,mensajes_enviados
0,1000,12,11
1,1001,8,30
2,1001,9,44
3,1001,10,53
4,1001,11,36
...,...,...,...
1801,1496,9,21
1802,1496,10,18
1803,1496,11,13
1804,1496,12,11


In [35]:
# Calcula el volumen del tráfico de Internet usado por cada usuario al mes. Guarda el resultado.
ith_mth = internet.groupby(["user_id","month"])["gb_used"].sum().reset_index()
ith_mth


,user_id,month,gb_used
0,1000,12,1.90147
1,1001,8,6.91915
2,1001,9,13.31482
3,1001,10,22.33049
4,1001,11,18.50430
...,...,...,...
2272,1498,12,23.13769
2273,1499,9,12.98476
2274,1499,10,19.49243
2275,1499,11,16.81383


[Junta los datos agregados en un DataFrame para que haya un registro que represente lo que consumió un usuario único en un mes determinado.]

In [62]:
# Fusiona los datos de llamadas, minutos, mensajes e Internet con base en user_id y month
usr_mth_cons = cals_per_user.merge(msj_mth, on= ["user_id","month"]).merge(ith_mth, on= ["user_id","month"])

usr_mth_cons


,user_id,month,minutos_consumidos,total_llamadas,Mensajes enviados,gb_used
0,1000,12,124,16,11,1.90147
1,1001,8,182,27,30,6.91915
2,1001,9,315,49,44,13.31482
3,1001,10,393,65,53,22.33049
4,1001,11,426,64,36,18.50430
...,...,...,...,...,...,...
1754,1496,9,414,52,21,16.38927
1755,1496,10,317,46,18,14.28736
1756,1496,11,308,39,13,8.54736
1757,1496,12,294,40,11,16.43899


In [ ]:
# Añade la información de la tarifa
users[["plan","user_id"]].merge(usr_mth_cons, on= "user_id") 


[Calcula los ingresos mensuales por usuario (resta el límite del paquete gratuito del número total de llamadas, mensajes de texto y datos; multiplica el resultado por el valor del plan de llamadas; añade la tarifa mensual en función del plan de llamadas). Nota: Dadas las condiciones del plan, ¡esto podría no ser tan trivial como un par de líneas! Así que no pasa nada si dedicas algo de tiempo a ello.]

In [ ]:
# Calcula el ingreso mensual para cada usuario



## Estudia el comportamiento de usuario

[Calcula algunas estadísticas descriptivas para los datos agregados y fusionados que nos sean útiles y que muestren un panorama general captado por los datos. Dibuja gráficos útiles para facilitar la comprensión. Dado que la tarea principal es comparar las tarifas y decidir cuál es más rentable, las estadísticas y gráficas deben calcularse por tarifa.]

[En los comentarios hallarás pistas relevantes para las llamadas, pero no las hay para los mensajes e Internet. Sin embargo, el principio del estudio estadístico que se aplica para ellos es el mismo que para las llamadas.]

### Llamadas

In [ ]:
# Compara la duración promedio de llamadas por cada plan y por cada mes. Traza un gráfico de barras para visualizarla.



In [ ]:
# Compara el número de minutos mensuales que necesitan los usuarios de cada plan. Traza un histograma.



[Calcula la media y la variable de la duración de las llamadas para averiguar si los usuarios de los distintos planes se comportan de forma diferente al realizar sus llamadas.]

In [ ]:
# Calcula la media y la varianza de la duración mensual de llamadas.



In [ ]:
# Traza un diagrama de caja para visualizar la distribución de la duración mensual de llamadas



[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a las llamadas. ¿Su comportamiento varía en función del plan?]

### Mensajes

In [ ]:
# Comprara el número de mensajes que tienden a enviar cada mes los usuarios de cada plan



In [ ]:
# Compara la cantidad de tráfico de Internet consumido por usuarios por plan



[Elabora las conclusiones sobre el comportamiento de los usuarios con respecto a los mensajes. ¿Su comportamiento varía en función del plan?]

### Internet

[Elabora las conclusiones sobre cómo los usuarios tienden a consumir el tráfico de Internet. ¿Su comportamiento varía en función del plan?]

## Ingreso

[Del mismo modo que has estudiado el comportamiento de los usuarios, describe estadísticamente los ingresos de los planes.]

[Elabora las conclusiones sobre cómo difiere el ingreso entre los planes.]

## Prueba las hipótesis estadísticas

[Prueba la hipótesis de que son diferentes los ingresos promedio procedentes de los usuarios de los planes de llamada Ultimate y Surf.]

[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]

In [ ]:
# Prueba las hipótesis



[Prueba la hipótesis de que el ingreso promedio de los usuarios del área NY-NJ es diferente al de los usuarios de otras regiones.]

[Elabora las hipótesis nula y alternativa, escoge la prueba estadística, determina el valor alfa.]

In [ ]:
# Prueba las hipótesis



## Conclusión general

[En esta sección final, enumera tus conclusiones importantes. Asegúrate de que estas abarquen todas las decisiones (suposiciones) importantes que adoptaste y que determinaron la forma elegida para procesar y analizar los datos.]